In [2]:
%matplotlib inline
from solvers import solver
import numpy as np
import qtools as qt
import tucker3d as tuck

Vendor:  Continuum Analytics, Inc.
Package: mkl
Message: trial mode expires in 28 days
openbabel not found in path, switching to PyQuante backend
libint extension not found, switching to normal ERI computation


In [17]:
name = 'H2'
molecule = qt.molecule(name)

gradients_005 = []
gradients_01 = []
gradients_02 = []
gradients_03 = []
gradients_05 = []
sols = []
for n in [2**11, 2**12, 2**13, 2**14]:
    sol = solver(molecule, method='hf', mixing=2, eps=1e-11,
                 maxiter=60, meshsize=n, boxsize=12.0)
    
    sol.solve()
    gradients_005.append(gradient_diatomic(sol, smoothing=0.005))
    gradients_01.append(gradient_diatomic(sol, smoothing=0.01))
    gradients_02.append(gradient_diatomic(sol, smoothing=0.02))
    gradients_03.append(gradient_diatomic(sol, smoothing=0.03))
    gradients_05.append(gradient_diatomic(sol, smoothing=0.05))
    
    print gradients

SCF iteration - done
0.0197141170502
(6, 6, 6)
Iteration 0 accuracy = 1.31e-02
Iteration 1 accuracy = 1.30e-02
Iteration 2 accuracy = 3.12e-03
Iteration 3 accuracy = 3.21e-04
Iteration 4 accuracy = 5.06e-04
Iteration 5 accuracy = 1.69e-04
Iteration 6 accuracy = 8.45e-06
Iteration 7 accuracy = 1.66e-05
Iteration 8 accuracy = 7.80e-06
Iteration 9 accuracy = 1.23e-06
Iteration 10 accuracy = 4.13e-07
Iteration 11 accuracy = 3.09e-07
Iteration 12 accuracy = 7.59e-08
Iteration 13 accuracy = 6.02e-09
Iteration 14 accuracy = 1.15e-08
Iteration 15 accuracy = 3.98e-09
Iteration 16 accuracy = 2.68e-10
Iteration 17 accuracy = 3.57e-10
Iteration 18 accuracy = 1.74e-10
Iteration 19 accuracy = 3.13e-11
Iteration 20 accuracy = 1.23e-11
Iteration 21 accuracy = 4.60e-12
Iteration 22 accuracy = 1.01e-11
Iteration 23 accuracy = 5.98e-12
Iteration 24 accuracy = 1.67e-11
Iteration 25 accuracy = 1.49e-11
Iteration 26 accuracy = 4.21e-13
Iteration 27 accuracy = 1.73e-12
Process converged with 27 iterations
Fu

In [18]:
print gradients_005
print gradients_01
print gradients_02
print gradients_03
print gradients_05

[(-0.008630454311897684+0j), (-0.0039147004959592113+0j), (-0.0054355179647423979+0j), (-0.0053909277324910576+0j)]
[(-0.012554897329275949+0j), (-0.0053293286139687313+0j), (-0.0053910828580464587+0j), (-0.0053908552800010501+0j)]
[(-0.0057241142278958712+0j), (-0.0053919304136270663+0j), (-0.0053908899668183574+0j), (-0.0053906296612981075+0j)]
[(-0.0054103746808907838+0j), (-0.0053922927135598986+0j), (-0.0053902953560241995+0j), (-0.0053905838102741299+0j)]
[(-0.0054030852009102626+0j), (-0.0053919112968487948+0j), (-0.0053912818647459115+0j), (-0.0053906743733432849+0j)]


In [19]:
def aitken(a2, a1, a0):
    return a2 - (a2-a1)**2/(a2-2*a1+a0)

print aitken(-0.005391281864745, -0.005391911296848, -0.0054030852009102)

-0.005391244292


In [16]:
print gradients_005
print gradients_01
print gradients_02
print gradients_03
print gradients_05

[(0.013611258586343822+0j), (0.01834078461823474+0j), (0.016818663663885292+0j), (0.016863542738754123+0j)]
[(0.009678550812098119+0j), (0.016924232130068051+0j), (0.016863169364275121+0j), (0.016863614213515843+0j)]
[(0.016525329371806674+0j), (0.016861598808606437+0j), (0.016863362213528632+0j), (0.016863839950110759+0j)]
[(0.016839854768045392+0j), (0.016861233749823568+0j), (0.016863957803458673+0j), (0.01686388528247118+0j)]
[(0.016847154796587716+0j), (0.016861612600190978+0j), (0.016862966227996601+0j), (0.016863791472206624+0j)]


In [9]:
gradient_diatomic(sol, smoothing=0.01)

(0.01686347827421647+0j)

In [12]:
gradient_diatomic(sol, smoothing=0.005)

(0.016863284839337245+0j)

In [33]:
from PyQuante.Molecule import Molecule
from PyQuante.dft import dft
h2 = Molecule('h2',[(1,(0,0,0)),(1,(1.4,0,0))])

In [35]:
en,orbe,orbs = dft(h2,functional='LDA')

Final LDA energy for system h2 is -1.135372


In [8]:
gradient_diatomic(sol, smoothing=0.2)

(0.016863768003298751+0j)

In [14]:
2*a - b

(0.00070546547821748185+0j)

In [4]:
2*a - b

(-0.0060777468614038321+0j)

In [16]:
x = sol.params.grid

In [17]:
tuck.dot(sol.rho, tuck.ones(sol.rho.n))*(x[1]-x[0])**3

0.99999476156725564

In [4]:
from scipy.special import erf

def gradient_diatomic(self, smoothing=0.01):

        # pot squared
        #tensor_x = charge*tuck.round(vec[0]*tuck.ones((N, N, N)) - tensor_x, self.params.eps)
        #qt.pots.coulomb(x, vec, self.params.ind, self.params.eps, beta=3.0)
        #d_coulomb -= tuck.cross.multifun([tensor_x, pot_squared], self.params.eps, lambda x: x[0]*x[1])

        molecule = self.molecule
        x = self.params.grid
        h = x[1] - x[0]
        N = len(self.params.grid)
        Norb = self.molecule.orbitals
        num_atoms = self.molecule.num_atoms
        
        i = 0
        vec = molecule.atoms[i].rad
        charge = molecule.atoms[i].charge
        def fun_coulomb_squared((i, j, k)):
            r = np.sqrt((vec[0] - x[i])**2 + (vec[1] - x[j])**2 + (vec[2] - x[k])**2)
            return charge*(x[i] - vec[0]) / r * (-smoothed_coulomb_derivative(r/smoothing))/smoothing**2
        d_coulomb = tuck.cross.cross3d(fun_coulomb_squared, N, self.params.eps)
        d_coulomb = tuck.round(d_coulomb, self.params.eps)
        

        self.get_rho()
        
        dE_nuc = 0.0
        j = 1
        i = 0
        vec_i = molecule.atoms[i].rad
        charge_i = molecule.atoms[i].charge
        vec_j = molecule.atoms[j].rad
        charge_j = molecule.atoms[j].charge
        dE_nuc = (dE_nuc + (vec_i[0] - vec_j[0])*charge_i*charge_j/
                  np.sqrt((vec_i[0] - vec_j[0])**2 + (vec_i[1] - vec_j[1])**2 + (vec_i[2] - vec_j[2])**2)**3)


        return tuck.dot(d_coulomb, 2*self.rho)*(x[1]-x[0])**3 + dE_nuc
    
def smoothed_coulomb_derivative(r):
        if r>=6:
            return -1.0/r**2
        elif r<6 and r>=0.1:
            return 2*np.exp(-r**2)/np.sqrt(np.pi)/r - erf(r)/r**2 - 1.0/3/np.sqrt(np.pi) * (2*r*np.exp(-r**2) + 128*r*np.exp(-4*r**2))
        else:
            return -4.0/3*r + 4.0/5*r**3 - 2.0/7*r**5 + 2.0/27*r**7 - 1.0/66*r**9 - 1.0/3/np.sqrt(np.pi) * (2*r*np.exp(-r**2) + 128*r*np.exp(-4*r**2))

In [17]:
from scipy.special import erf

def gradient_diatomic(self, smoothing=0.01):

        # pot squared
        #tensor_x = charge*tuck.round(vec[0]*tuck.ones((N, N, N)) - tensor_x, self.params.eps)
        #qt.pots.coulomb(x, vec, self.params.ind, self.params.eps, beta=3.0)
        #d_coulomb -= tuck.cross.multifun([tensor_x, pot_squared], self.params.eps, lambda x: x[0]*x[1])

        molecule = self.molecule
        x = self.params.grid
        h = x[1] - x[0]
        N = len(self.params.grid)
        Norb = self.molecule.orbitals
        num_atoms = self.molecule.num_atoms
        
        i = 0
        vec = molecule.atoms[i].rad
        charge = molecule.atoms[i].charge
        def fun_coulomb_squared((i, j, k)):
            r = np.sqrt((vec[0] - x[i])**2 + (vec[1] - x[j])**2 + (vec[2] - x[k])**2)
            return charge*(x[i] - vec[0]) / r * (-smoothed_coulomb_derivative(r/smoothing))/smoothing**2
        d_coulomb = tuck.cross.cross3d(fun_coulomb_squared, N, self.params.eps)
        d_coulomb = tuck.round(d_coulomb, self.params.eps)
        

        self.get_rho()
        
        dE_nuc = 0.0
        j = 1
        i = 0
        vec_i = molecule.atoms[i].rad
        charge_i = molecule.atoms[i].charge
        vec_j = molecule.atoms[j].rad
        charge_j = molecule.atoms[j].charge
        dE_nuc = (dE_nuc + (vec_i[0] - vec_j[0])*charge_i*charge_j/
                  np.sqrt((vec_i[0] - vec_j[0])**2 + (vec_i[1] - vec_j[1])**2 + (vec_i[2] - vec_j[2])**2)**3)


        return tuck.dot(d_coulomb, 2*tuck.ones(d_coulomb.n))*(x[1]-x[0])**3 #+ dE_nuc
    
def smoothed_coulomb_derivative(r):
        if r>=6:
            return -1.0/r**2
        elif r<6 and r>=0.1:
            return 2*np.exp(-r**2)/np.sqrt(np.pi)/r - erf(r)/r**2 - 1.0/3/np.sqrt(np.pi) * (2*r*np.exp(-r**2) + 128*r*np.exp(-4*r**2))
        else:
            return -4.0/3*r + 4.0/5*r**3 - 2.0/7*r**5 + 2.0/27*r**7 - 1.0/66*r**9 - 1.0/3/np.sqrt(np.pi) * (2*r*np.exp(-r**2) + 128*r*np.exp(-4*r**2))

In [10]:
def gradient_diatomic(self):

        # pot squared
        #tensor_x = charge*tuck.round(vec[0]*tuck.ones((N, N, N)) - tensor_x, self.params.eps)
        #qt.pots.coulomb(x, vec, self.params.ind, self.params.eps, beta=3.0)
        #d_coulomb -= tuck.cross.multifun([tensor_x, pot_squared], self.params.eps, lambda x: x[0]*x[1])

        molecule = self.molecule
        x = self.params.grid
        h = x[1] - x[0]
        N = len(self.params.grid)
        Norb = self.molecule.orbitals
        num_atoms = self.molecule.num_atoms

        i = 0
        vec = molecule.atoms[i].rad
        charge = molecule.atoms[i].charge
        def fun_coulomb_squared((i, j, k)):
            return charge*(x[i] - vec[0]) / np.sqrt((vec[0] - x[i])**2 + (vec[1] - x[j])**2 + (vec[2] - x[k])**2)**(3.0)
        d_coulomb = tuck.cross.cross3d(fun_coulomb_squared, N, self.params.eps)
        d_coulomb = tuck.round(tuck.real(d_coulomb), self.params.eps)

        self.get_rho()
        #d_coulomb = tuck.cross.multifun([self.rho, d_coulomb], self.params.eps, lambda x: x[0]*x[1])
        
        
        dE_nuc = 0.0
        j = 1
        i = 0
        vec_i = molecule.atoms[i].rad
        charge_i = molecule.atoms[i].charge
        vec_j = molecule.atoms[j].rad
        charge_j = molecule.atoms[j].charge
        dE_nuc = (dE_nuc + (vec_i[0] - vec_j[0])*charge_i*charge_j/
                  np.sqrt((vec_i[0] - vec_j[0])**2 + (vec_i[1] - vec_j[1])**2 + (vec_i[2] - vec_j[2])**2)**3)

        return tuck.dot(d_coulomb, 2*self.rho)*(x[1]-x[0])**3 + dE_nuc